# Mackerel Data Analysis
Roughly following the structure of the QIIME 2 "moving pictures" tutorial -- this focuses on just getting the data ready for analysis in Songbird and Qurro.

The data is from [study 11721 on Qiita](https://qiita.ucsd.edu/study/description/11721), and is associated with a manuscript currently in preparation (Minich et al. 2019, preprint available on bioRxiv [here](https://www.biorxiv.org/content/10.1101/721555v1)).

The input data in this notebook is 150nt sOTU data, corresponding to artifact ID `56427` on Qiita. These data were processed on Qiita using QIIME 1.9.1 (`Split libraries FASTQ` and `Trimming`), then denoised using Deblur 1.1.0. The `reference-hit` BIOM table and FASTA file were used as the starting point for this analysis.

## 0. Setting up
### 0.1. Declare some environment variables

In [1]:
# Input Data Locations (trimmed-to-150-nt and deblurred BIOM table and sequences,
# as well as sample metadata)
%env INPUT_BIOM_TABLE_PATH=/projects/qiita_data/BIOM/56427/reference-hit.biom
%env INPUT_REP_SEQS_PATH=/projects/qiita_data/BIOM/56427/reference-hit.seqs.fa
%env INPUT_SAMPLE_METADATA_PATH=/projects/qiita_data/templates/11721_prep_4638_qiime_20190722-104633.txt

# Output directory (will contain all .qza and .qzv files generated by this analysis)
%env OUTPUT_DIRECTORY=/home/mfedarko/analyses/qurro-mackerel-analysis/AnalysisOutput

env: INPUT_BIOM_TABLE_PATH=/projects/qiita_data/BIOM/56427/reference-hit.biom
env: INPUT_REP_SEQS_PATH=/projects/qiita_data/BIOM/56427/reference-hit.seqs.fa
env: INPUT_SAMPLE_METADATA_PATH=/projects/qiita_data/templates/11721_prep_4638_qiime_20190722-104633.txt
env: OUTPUT_DIRECTORY=/home/mfedarko/analyses/qurro-mackerel-analysis/AnalysisOutput


### 0.2. Move into the output directory

In [2]:
import os
odir = os.environ["OUTPUT_DIRECTORY"]
os.chdir(odir)
print("Moved into output directory: {}".format(odir))

Moved into output directory: /home/mfedarko/analyses/qurro-mackerel-analysis/AnalysisOutput


### 0.3. Get information about the current QIIME 2 environment
(For future reference.)

In [3]:
!qiime info

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.
System versions
Python version: 3.6.7
QIIME 2 release: 2019.7
QIIME 2 version: 2019.7.0
q2cli version: 2019.7.0

Installed plugins
alignment: 2019.7.0
composition: 2019.7.0
cutadapt: 2019.7.0
dada2: 2019.7.0
deblur: 2019.7.0
demux: 2019.7.0
diversity: 2019.7.0
emperor: 2019.7.0
feature-classifier: 2019.7.0
feature-table: 2019.7.0
fragment-insertion: 2019.7.0
gneiss: 2019.7.0
longitudinal: 2019.7.0
metadata: 2019.7.0
phylogeny: 2019.7.0
quality-control: 2019.7.0
quality-filter: 2019.7.0
qurro: 0.4.0
sample-classifier: 2019.7.0
songbird: 0.9.0
taxa: 2019.7.0
types: 2019.7.0
vsearch: 2019.7.0

Application config directory
/home/mfedarko/.config/q2cli

Getting help
To get help with QIIME 2, visit https://qiime2.org


## 1. Import data into QIIME 2 artifacts
See [the QIIME 2 documentation on importing data](https://docs.qiime2.org/2019.4/tutorials/importing/) for context on why this is necessary and useful.

### 1.1. Import the study's data
Note that this dataset doesn't just contain data about the microbiota of pacific chub mackerel: it also contains samples taken from other species of fish, as well as well as environmental samples. We'll filter some of these samples out of the dataset soon.

In [4]:
!qiime tools import \
    --type "FeatureTable[Frequency]" \
    --input-path $INPUT_BIOM_TABLE_PATH \
    --output-path table-unfiltered.qza
!qiime tools import \
    --type "FeatureData[Sequence]" \
    --input-path $INPUT_REP_SEQS_PATH \
    --output-path rep-seqs-unfiltered.qza

Imported /projects/qiita_data/BIOM/56427/reference-hit.biom as BIOMV210DirFmt to table-unfiltered.qza
Imported /projects/qiita_data/BIOM/56427/reference-hit.seqs.fa as DNASequencesDirectoryFormat to rep-seqs-unfiltered.qza


#### 1.1.1. Summarize the imported study table and representative sequence data
This gives us information about the number of samples and sequences present in these files. It's useful for sanity-checking the filtering that will be done in the next section.

In [5]:
!qiime feature-table summarize \
    --i-table table-unfiltered.qza \
    --o-visualization table-unfiltered-summary.qzv \
    --m-sample-metadata-file $INPUT_SAMPLE_METADATA_PATH

!qiime feature-table tabulate-seqs \
    --i-data rep-seqs-unfiltered.qza \
    --o-visualization rep-seqs-unfiltered-summary.qzv

Saved Visualization to: table-unfiltered-summary.qzv
Saved Visualization to: rep-seqs-unfiltered-summary.qzv


### 1.2. Import Greengenes 13_8 99% reference data
See [DeSantis et al. 2006](https://aem.asm.org/content/72/7/5069.short) and [McDonald et al. 2012](https://www.nature.com/articles/ismej2011139?report=reader). We'll use this in the "Taxonomic classification" section below.

In [6]:
!qiime tools import \
    --type 'FeatureData[Sequence]' \
    --input-path /databases/gg/13_8/rep_set/99_otus.fasta \
    --output-path gg_13_8_99_otus.qza

!qiime tools import \
    --type 'FeatureData[Taxonomy]' \
    --input-format HeaderlessTSVTaxonomyFormat \
    --input-path /databases/gg/13_8/taxonomy/99_otu_taxonomy.txt \
    --output-path gg_13_8_99_taxonomy.qza

Imported /databases/gg/13_8/rep_set/99_otus.fasta as DNASequencesDirectoryFormat to gg_13_8_99_otus.qza
Imported /databases/gg/13_8/taxonomy/99_otu_taxonomy.txt as HeaderlessTSVTaxonomyFormat to gg_13_8_99_taxonomy.qza


## 2. Filter the study's data
In particular, we'll filter the feature table (and representative sequences) to only contain samples that satisfy both of the following two conditions:

1. have a `host_common_name` of `pacific chub mackerel` OR have a `sample_type_body_site` of `sea water`
2. contain at least 1,362 sequences

### 2.1. Why do we filter to just pacific chub mackerel and sea water samples?
If you examine `table-unfiltered-summary.qzv` (in particular the "Interactive Sample Detail" tab), you should see that only 1,173 / 1,530 samples have a `host_common_name` of `pacific chub mackerel`. We're going to look at how samples taken from various body sites of these mackerel differ from environmental samples (in particular, just samples taken from sea water).

In order to perform this analysis, we filter the table to just samples where `host_common_name` is `pacific chub mackerel` *or* samples where `sample_type_body_site` is `sea water`.

### 2.2. Why do we filter out samples with less than 1,362 sequences?
This isn't rarefaction—the remaining samples have all their sequences preserved—but samples with less than this number of sequences are removed from the analysis from here on down.

The "1,362" number is based on this data's corresponding study's results from using the KatharoSeq protocol: see [Minich et al. 2018](https://msystems.asm.org/content/3/3/e00218-17.abstract) for a description of how KatharoSeq works, and Minich et al. 2019 (see intro section of this analysis for details) for details on how this number was determined.

In [7]:
!qiime feature-table filter-samples \
    --i-table table-unfiltered.qza \
    --m-metadata-file $INPUT_SAMPLE_METADATA_PATH \
    --p-where "host_common_name='pacific chub mackerel' OR sample_type_body_site='sea water'" \
    --p-min-frequency 1362 \
    --o-filtered-table table.qza

# Filter rep-seqs-unfiltered.qza to only include sequences present in the now-filtered table (table.qza).
!qiime feature-table filter-seqs \
    --i-table table.qza \
    --i-data rep-seqs-unfiltered.qza \
    --o-filtered-data rep-seqs.qza

Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza


### 2.3. Summarize the filtered data
This will let us double-check that the filtering above was done properly.

In [8]:
!qiime feature-table summarize \
    --i-table table.qza \
    --o-visualization table-summary.qzv \
    --m-sample-metadata-file $INPUT_SAMPLE_METADATA_PATH

!qiime feature-table tabulate-seqs \
    --i-data rep-seqs.qza \
    --o-visualization rep-seqs-summary.qzv

Saved Visualization to: table-summary.qzv
Saved Visualization to: rep-seqs-summary.qzv


## 3. Taxonomic classification
You don't *need* taxonomy information (i.e. feature metadata) to run Songbird or Qurro. However, having this information available is extremely useful in interpreting a Qurro visualization -- this is why we'll perform taxonomic classification on our dataset's features.

We're going to do this taxonomic classification using VSEARCH (see [Rognes et al. 2016](https://peerj.com/articles/2584/)) and based on the Greengenes 13_8 99% database (see above for citations).

In [9]:
!qiime feature-classifier classify-consensus-vsearch \
    --i-query rep-seqs.qza \
    --i-reference-reads gg_13_8_99_otus.qza \
    --i-reference-taxonomy gg_13_8_99_taxonomy.qza \
    --o-classification taxonomy.qza

Saved FeatureData[Taxonomy] to: taxonomy.qza


## 4. Filter features with taxonomy containing the text `mitochondria` and `chloroplast`
Based on [this QIIME 2 tutorial](https://docs.qiime2.org/2019.7/tutorials/filtering/#taxonomy-based-filtering-of-tables-and-sequences). There are actually a decent amount of Chloroplast features in this dataset (which makes sense, since these are marine samples), but since they're irrelevant to our analysis we filter them out.

In [15]:
!qiime taxa filter-table \
    --i-table table.qza \
    --i-taxonomy taxonomy.qza \
    --p-exclude "mitochondria,chloroplast" \
    --o-filtered-table table-no-mitochondria-or-chloroplast.qza

Saved FeatureTable[Frequency] to: table-no-mitochondria-or-chloroplast.qza


## 5. Run Songbird
This will generate feature differentials, which we'll visualize in Qurro.
For details on what Songbird does and how it works, please see [Songbird's GitHub page](https://github.com/biocore/songbird/), as well as [Morton and Marotz et al. 2019](https://www.nature.com/articles/s41467-019-10656-5).

### 5.1. Why do we manually set certain Songbird parameters?
These parameters were chosen based on consulting Tensorboard to ensure that they resulted in a reasonable model fit.

#### `--p-formula`
This parameter is used by Songbird to determine what sample metadata fields should be used as covariates when generating differentials. Here, we generate differentials relative to the `sample_type_body_site` field (using the `sea water` values of this field as a reference), but there are of course plenty of other options for fields that could be used here if we'd like to ask different questions about this data.

#### `--p-epochs`, `--p-learning-rate`, `--p-batch-size`
These parameters influence Songbird's runtime:

- We've increased `--p-epochs` from the default of 1,000 to 5,000 to make Songbird run for a bit longer (we're working with a fairly large dataset).
- We've decreased `--p-learning-rate` from the default of 0.001 to 0.0001 to similarly increase Songbird's run time.
- We've increased `--p-batch-size` from the default of 5 to 10 to make Songbird process a larger amount of samples at once in each iteration. Since our samples fall into six "categories" (the five mackerel body sites, plus sea water samples), using a larger batch size (that stands a better chance of reflecting this diversity) makes sense.

#### `--p-num-random-test-examples`
Quoting Songbird's documentation again, this is "\[the number\] of random samples to hold out for cross-validation if `training-column` is not specified." The default for this is 5 (i.e. use just 5 samples for cross-validation); since we have the luxury of having a lot of samples in this dataset, we can afford to hold out more samples. This is why we've increased this to 30 samples.

#### `--p-summary-interval`
This is the frequency (in seconds) of how often Songbird saves model fitting statistics in the `--o-regression-stats` output artifact. More frequent measurements will help us more accurately diagnose if Songbird's model is fitting reasonably to the dataset. See [this section of Songbird's README](https://github.com/biocore/songbird#interpreting-model-fitting) for details on model fitting.

In [25]:
!qiime songbird multinomial \
    --i-table table-no-mitochondria-or-chloroplast.qza \
    --m-metadata-file $INPUT_SAMPLE_METADATA_PATH \
    --p-formula "C(sample_type_body_site, Treatment('sea water'))" \
    --p-epochs 5000 \
    --p-learning-rate 0.0001 \
    --p-num-random-test-examples 30 \
    --p-batch-size 10 \
    --p-summary-interval 1 \
    --verbose \
    --o-differentials songbird-differentials.qza \
    --o-regression-stats songbird-regression-stats.qza \
    --o-regression-biplot songbird-regression-biplot.qza

W0926 20:59:14.648060 140270108981056 deprecation_wrapper.py:119] From /home/mfedarko/.conda/envs/qiime2-2019.7/lib/python3.6/site-packages/songbird/q2/_method.py:50: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2019-09-26 20:59:14.648933: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2019-09-26 20:59:14.664770: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2600035000 Hz
2019-09-26 20:59:14.665349: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e0e5122ae0 executing computations on platform Host. Devices:
2019-09-26 20:59:14.665399: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
OMP: Info #212:

 16%|█████▎                            | 46669/300000 [00:43<03:46, 1118.22it/s]2019-09-26 20:59:57.957213: I tensorflow/core/profiler/lib/profiler_session.cc:174] Profiler session started.


 32%|██████████▋                       | 94715/300000 [01:25<03:02, 1125.97it/s]2019-09-26 21:00:40.981924: I tensorflow/core/profiler/lib/profiler_session.cc:174] Profiler session started.


 48%|███████████████▋                 | 142503/300000 [02:09<02:21, 1109.42it/s]2019-09-26 21:01:24.001544: I tensorflow/core/profiler/lib/profiler_session.cc:174] Profiler session started.


 63%|████████████████████▉            | 190142/300000 [02:52<01:41, 1086.19it/s]2019-09-26 21:02:07.021710: I tensorflow/core/profiler/lib/profiler_session.cc:174] Profiler session started.


 79%|██████████████████████████       | 236388/300000 [03:35<00:58, 1080.66it/s]2019-09-26 21:02:50.043864: I tensorflow/core/profiler/lib/profiler_session.cc:174] Profiler session started.


 95%|███████████████████████████████▏ | 283903/300000 [04:18<00:14, 1113.06it/s]2019-09-26 21:03:33.065696: I tensorflow/core/profiler/lib/profiler_session.cc:174] Profiler session started.


100%|█████████████████████████████████| 300000/300000 [04:32<00:00, 1100.18it/s]
Saved FeatureData[Differential] to: songbird-differentials.qza
Saved SampleData[SongbirdStats] to: songbird-regression-stats.qza
Saved PCoAResults % Properties('biplot') to: songbird-regression-biplot.qza


### 5.2. Visualize Songbird model fitting statistics
For more information, check out [this section of the Songbird README](https://github.com/biocore/songbird#interpreting-model-fitting). Note that (as of writing) the version of Songbird used here is a bit older than my in-development version of it, so the QIIME 2 summary statistics here might look a bit funky. Essentially, the ordering of plots is switched from how it *will* be in future versions of these summaries: the `Loglikehood` _[sic]_ plot on the top corresponds to loss, a.k.a. the bottom plot in Tensorflow.

In [26]:
!qiime songbird summarize-single \
    --i-feature-table table-no-mitochondria-or-chloroplast.qza \
    --i-regression-stats songbird-regression-stats.qza \
    --o-visualization songbird-regression-summary.qzv

Saved Visualization to: songbird-regression-summary.qzv


## 6. Run Qurro!
We're doing this using Qurro v0.4.0. Note that the version of the "mackerel demo" up on [Qurro's website](https://biocore.github.io/qurro) will be updated as Qurro itself is updated (so although the underlying data should remain the same, the visualization interface might look a bit different/contain a few more features in a few months).

In [27]:
!qiime qurro differential-plot \
    --i-table table-no-mitochondria-or-chloroplast.qza \
    --i-ranks songbird-differentials.qza \
    --m-sample-metadata-file $INPUT_SAMPLE_METADATA_PATH \
    --m-feature-metadata-file taxonomy.qza \
    --verbose \
    --o-visualization qurro-plot.qzv

20973 feature(s) in the BIOM table were not present in the feature rankings.
These feature(s) have been removed from the visualization.
1247 sample(s) in the sample metadata file were not present in the BIOM table.
These sample(s) have been removed from the visualization.
Saved Visualization to: qurro-plot.qzv
